In [7]:
import keras
from keras import layers
from keras.preprocessing import image

import os

import numpy as np

from imageio import imwrite

from tqdm import tqdm_notebook

https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.5-introduction-to-gans.ipynb

Deep Convolution GAN - DCGAN

Структура сети:
* Сеть generator отображает векторы с формой (размерность_скрытого_простаранства) в изображения с формой (32, 32, 3).
Размерность изображения соответсвует размерам образцов из датасета CIFAR10
* Сеть discriminator отображает изображения с формой (32, 32, 3) в оценку вероятности того, что изображение является настоящим.
* Сеть gan объединяет генератор и дискриминатор gan(x) = discriminator(generator(x)). То есть сеть отображает скрытое простарнство векторов в оценку их реализма.

Дискриминатор будет обучен на изображениях с метками настоящее\поддельное. <br>
Для обучения генератора мы используем градиенты весов в отношении потерь модели gan. То есть на каждом шаге мы будем смещать веса генератора в направлении увеличения вероятности классификации изображений, декодированных как "настоящие". <br>
Проще говоря, будем учить генератор обманывать дискриминатор, повышая качество сгенерированных изображений.

**Генератор** <br>
Преобразует вектор из срытого пространства в изображение - кандидат.

In [8]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape = (latent_dim,))

# Преобразование входа в карту признаков 16х16 со 128 каналами
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# Увеличиваем разрешение с 16х16 до 32х32
x = layers.Conv2D(256, 5, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides = 2, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding = 'same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation = 'tanh', padding = 'same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 256)       1048

**Дискриминатор**<br>
Принимает изображение-кандидат и относит его к одному из двух классов: "подделка" или "настоящее"

In [9]:
discriminator_input = layers.Input(shape = (height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation = 'sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(lr = 0.0008, 
                                                   clipvalue = 1.0, # градиентная обрезка (клиппинг)
                                                   decay = 1e-8) # затухание скорости обучения, для стабилизации
discriminator.compile(optimizer = discriminator_optimizer, 
                     loss = 'binary_crossentropy')

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 6, 6, 128)         0   

**Состязательная сеть**<br>
Объединятет генератор и дискриминатор

In [10]:
discriminator.trainable = False #Заморозка весов дискриминатора

gan_input = keras.Input(shape = (latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr = 0.0004, clipvalue = 1.0, decay = 1e-8)
gan.compile(optimizer = gan_optimizer, loss = 'binary_crossentropy')

**Загрузка данных и обучение**

In [11]:
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 3] # Выбираем класс изображений "птицы"

x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32')/255.

Цикл обучения:
* Извлечь случайные точки (случайный шум) из скрытого пространства.
* Создать изображение из генератора, использовав эти точки.
* Смешать сгенерированные изображения с настоящими.
* Обучить дискриминатор на этом наборе изображений, добавив к нему соответсвующие метки: "настоящее"/"подделка".
* Выбрать новые точки из сркытого пространства.
* Обучить gan, использовав эти случайные изображения, присвоив им метки "настоящее". Так как внутри gan веса дискриминатора заморожены, то в процессе обучения сместяться только веса генератора, в направлении получения от дискриминатора ответа "настоящее". Это научит генератор обманывать дискриминатор.  


In [12]:
iterations = 10000
batch_size = 20
save_dir = 'GAN (8.5)/'

start = 0
for step in tqdm_notebook(range(iterations)):
    #Выбор случайных точек из скрытого пространства
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim))
    #Декодируем точки в поддельное изображение
    generated_images = generator.predict(random_latent_vectors) 
    
    #Объединяем подделки с настоящими изображениями
    stop = start + batch_size
    real_images = x_train[start:stop]
    combined_images = np.concatenate([generated_images, real_images])
    
    #Сборка меток изображений
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape) #Добавление случайного шума в метки
    
    #Обучение дискриминатора
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    #Выбор случайных точек из скрытого пространства
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim))
    
    #Сборка меток изображений, которые всегда говорят "настоящее" (это ложь)
    misleading_targets = np.zeros((batch_size, 1))
    
    #Обучение генератора (через модель gan, то есть с замороженным дискриминатором)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    #Сохраняем изображения и текущую версию модели
    if step % 100 == 0: 
        gan.save_weights(os.path.join(save_dir,'gan.h5'))
        print('discriminator loss', d_loss)
        print('adversarial loss', a_loss)
        
        img = image.array_to_img(generated_images[0] * 255., scale = False)
        imwrite(os.path.join(save_dir, 'generated_' + str(step) + '.png'), img)
        
        img = image.array_to_img(real_images[0] * 255., scale = False)
        imwrite(os.path.join(save_dir, 'real_' + str(step) + '.png'), img)
    
    

discriminator loss 0.689222
adversarial loss 0.16639897


C:\Users\Grigoriy\.conda\envs\tf_cpu\lib\site-packages\keras\engine\training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


KeyboardInterrupt: 